In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD, Adam, Adagrad, RMSprop
from tensorflow.keras.models import load_model
import os
import time
from glob import glob
import cv2

In [ ]:
def model_def(view_model=False):
    
    K.clear_session()
    
    input_shape = (256, 256, 3)
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64, (3,3), padding="same", activation="relu", input_shape=input_shape),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, (3,3), padding="same", activation="relu"),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(32, (3,3), padding="same", activation="relu"),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(32, (3,3), padding="same", activation="relu"),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(16, (3,3), padding="same", activation="relu"),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(16, (3,3), padding="same", activation="relu"),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(16, (3,3), padding="same", activation="relu"),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation="relu"),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(12, activation="softmax")])
    
    if view_model:
        model.summary()
    
    return model

In [ ]:
def center_stdNorm(x):
    
    x_norm = np.zeros(x.shape)
    for i in range(len(x)):
        x_norm[i] = (x[i]-np.mean(x[i], keepdims=True))/(np.std(x[i], keepdims=True)+1e-6)
    
    return x_norm

In [ ]:
img_dir = 'base2'
dir_prefix = '*/'
file_prefix = '*.jpg'

dir_list =  glob(os.path.join(img_dir, dir_prefix))
img_list = []
labels_list = []

for i in range(np.size(dir_list)):
    for filename in glob(os.path.join(dir_list[i], file_prefix)):
        img = cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2RGB)
        img_list.append(img)
        labels_list.append(i)
        
x = np.array(img_list, dtype=np.uint8)
y = np.array(labels_list, dtype=np.uint8)

In [ ]:
test_size=0.1
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=test_size,
                                                    random_state=42,
                                                    stratify=y)

In [ ]:
seq = iaa.Sequential([
    iaa.Fliplr(1.0),
    #iaa.Flipud(1.0),
    iaa.Affine(translate_px={'x': (-30,30), 'y': (-30,30)}),
    iaa.Affine(scale={"x": (0.7, 1.3), "y": (0.7, 1.3)})
])

x_train_aug = seq(images=x_train)

x_train = np.concatenate((x_train, x_train_aug))
y_train = np.concatenate((y_train, y_train))

x_train_norm = center_stdNorm(x_train)
x_test_norm = center_stdNorm(x_test)

y_train = tf.keras.utils.to_categorical(y_train, 12)
y_test = tf.keras.utils.to_categorical(y_test, 12)

In [ ]:
K.clear_session()

model = model_def()

test_name = 'rmsprop_0.001_100_2'
model_path = os.path.join('trained_models/', test_name)

model.compile(optimizer=RMSprop(learning_rate=0.001, momentum=0.0),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

cp_callback = callback_func(model_path)

start_time = time.time()
history = model.fit(x = x_train_norm,
                    y = y_train,
                    batch_size=16,
                    epochs=100,
                    validation_data=(x_test_norm, y_test)
                    )
training_time = time.time() - start_time
     
print("\n--- Training time: %s seconds ---" % training_time)

val_loss, val_acc = model.evaluate(x=x_test_norm,
                                   y=y_test)

# Salvar o modelo
model.save(os.path.join(model_path, test_name+'.h5'))
print('Saved trained model at %s ' % model_path)

h = history.history
np.savez(os.path.join(model_path, test_name+'_history'), accuracy=h['accuracy'], loss=h['loss'], val_accuracy=h['val_accuracy'], val_loss=h['val_loss'])

In [ ]:
model.evaluate(x_test, steps=len(x_test))